In [1]:
print("STARTO")
import time
import os
start_time = time.time()

using_gpu = True
if not using_gpu:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
    os.environ["CUDA_VISIBLE_DEVICES"] = ""

from lib.detector.YOLO.train.trainer import Trainer
from lib.detector.YOLO import create_detector_model_from_trained
from lib.detector.YOLO import create_training_model_from_detector
from lib.detector.YOLO import Yolo

STARTO


Using TensorFlow backend.


In [2]:
from contextlib import contextmanager

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

In [3]:
from keras.models import load_model
pretrained = create_training_model_from_detector(load_model('./model_data/yolo.h5'))

/src/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
# pretrained_model = create_training_model_from_detector(pretrained)
pretrained_model = pretrained
for i in range(len(pretrained_model.layers)):
    pretrained_model.layers[i].trainable = False

In [5]:
# pretrained_model.layers[-1].trainable = True
# pretrained_model.layers[-4].trainable = True

In [6]:
labels = ['bicycle','bus','car','motorbike']

In [7]:
import numpy as np
import json
with open('voc_4.json', 'r') as fp:
    data = json.load(fp)
    train_imgs = data['train']
    valid_imgs = data['val']

In [8]:
hue = {}
for valid in valid_imgs:
    for obj in valid['object']:
        if obj['name'] not in hue:
            hue[obj['name']] = 1
        else:
            hue[obj['name']] +=1
hue

{'car': 515, 'bus': 137, 'motorbike': 162, 'bicycle': 153}

In [9]:
hue = {}
for train in train_imgs:
    for obj in train['object']:
        if obj['name'] not in hue:
            hue[obj['name']] = 1
        else:
            hue[obj['name']] +=1
hue

{'bicycle': 684, 'car': 1977, 'motorbike': 639, 'bus': 548}

In [10]:
tr = Trainer(model=pretrained_model, input_size=608, max_box_per_image=25, labels=labels, anchors=[0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 608, 608, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 608, 608, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 608, 608, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 608, 608, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [11]:
!rm -rf logs/log_voc_4
!mkdir logs/log_voc_4

In [12]:
tr.randomize_weight()

In [13]:
tr.train(
    tensorboard_log='/src/workspace/logs/v1',
    saved_weights_name='best_trained_v1.h5',
    train_imgs=train_imgs,
    batch_size=64,
    valid_imgs=valid_imgs,
    learning_rate=0.001,
    valid_times=1,
    train_times=1,
    debug=False
)

Epoch 1/104
96/96 [==============================] - 181s 2s/step - loss: 7.3528 - val_loss: 6.9153

Epoch 00001: val_loss improved from inf to 6.91533, saving model to voc_4.h5
Epoch 2/104
96/96 [==============================] - 161s 2s/step - loss: 6.9962 - val_loss: 6.8832

Epoch 00002: val_loss improved from 6.91533 to 6.88318, saving model to voc_4.h5
Epoch 3/104
96/96 [==============================] - 170s 2s/step - loss: 6.9586 - val_loss: 6.8692

Epoch 00003: val_loss improved from 6.88318 to 6.86918, saving model to voc_4.h5
Epoch 4/104
96/96 [==============================] - 174s 2s/step - loss: 6.9550 - val_loss: 6.8700

Epoch 00004: val_loss did not improve from 6.86918
Epoch 5/104
96/96 [==============================] - 171s 2s/step - loss: 8.7733 - val_loss: 10.4278

Epoch 00005: val_loss did not improve from 6.86918
Epoch 6/104
96/96 [==============================] - 171s 2s/step - loss: 9.1502 - val_loss: 8.5906

Epoch 00006: val_loss did not improve from 6.86918
E

In [14]:
from keras.models import save_model

In [15]:
yolo_model_best = create_detector_model_from_trained(load_model('best_trained_v1.h5', compile=False))
save_model(yolo_model_best, 'v1.h5', include_optimizer=False)

In [16]:
elapsed_time = time.time() - start_time
print(elapsed_time)

3187.154341697693
